http://rail.eecs.berkeley.edu/deeprlcourse/static/homeworks/hw1.pdf

In [1]:
import pickle
import keras
import numpy as np
import os

Using TensorFlow backend.


In [2]:
# env_name = 'RoboschoolHalfCheetah-v1'
env_name = "RoboschoolHumanoid-v1"
rollouts_dir = 'expert_data'
roll_out_file = os.path.join(rollouts_dir, env_name + '_dagger_2.pkl')
model_name = 'model_' + env_name + '_dagger_2'

In [3]:
# read roll-outs
with open(roll_out_file, 'rb') as f:
    roll_outs = pickle.loads(f.read())

In [4]:
print(roll_outs['observations'].shape)
print(roll_outs['actions'].shape)

(1240369, 44)
(1240369, 17)


In [5]:
from keras.layers import Input, Dense
from keras.models import Model
import math

def create_model(input_size, output_size):
    inputs = Input(shape=(input_size,))
    
    x = Dense(input_size * 2, activation='relu')(inputs)
#     x = keras.layers.Dropout(0.1)(x)
#     x = Dense(50, activation='relu')(x)
#     x = keras.layers.Dropout(0.2)(x)
#     x = Dense(20, activation='relu')(x)
    x = Dense(math.ceil(input_size / 2), activation='relu')(x)
    
    predictions = Dense(output_size)(x)
    
    return Model(inputs=inputs, outputs=predictions)

In [6]:
model = create_model(roll_outs['observations'].shape[1], roll_outs['actions'].shape[1])

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 44)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 88)                3960      
_________________________________________________________________
dense_2 (Dense)              (None, 22)                1958      
_________________________________________________________________
dense_3 (Dense)              (None, 17)                391       
Total params: 6,309
Trainable params: 6,309
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [9]:
X = roll_outs['observations']
Y = roll_outs['actions']

In [10]:
model.fit(X, Y, epochs=20, batch_size=256)

Epoch 1/20
1240369/1240369 [==============================] - 8s 7us/step - loss: 0.0320
Epoch 2/20
1240369/1240369 [==============================] - 8s 6us/step - loss: 0.0185
Epoch 3/20
1240369/1240369 [==============================] - 8s 7us/step - loss: 0.0166
Epoch 4/20
1240369/1240369 [==============================] - 8s 6us/step - loss: 0.0157
Epoch 5/20
1240369/1240369 [==============================] - 8s 6us/step - loss: 0.0151
Epoch 6/20
1240369/1240369 [==============================] - 8s 6us/step - loss: 0.0147
Epoch 7/20
1240369/1240369 [==============================] - 8s 6us/step - loss: 0.0144
Epoch 8/20
1240369/1240369 [==============================] - 8s 6us/step - loss: 0.0142
Epoch 9/20
1240369/1240369 [==============================] - 7s 6us/step - loss: 0.0140
Epoch 10/20
1240369/1240369 [==============================] - 7s 6us/step - loss: 0.0139
Epoch 11/20
1240369/1240369 [==============================] - 7s 6us/step - loss: 0.0138
Epoch 12/20
1240369

In [11]:
model.save_weights(model_name)

In [12]:
# model.load_weights(model_name)

In [13]:
# test our model with actual environment
import gym
import roboschool
env = gym.make(env_name)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: Environment '<class 'roboschool.gym_mujoco_walkers.RoboschoolHumanoid'>' has deprecated methods '_step' and '_reset' rather than 'step' and 'reset'. Compatibility code invoked. Set _gym_disable_underscore_compat = True to disable this behavior.


/data/apps/anaconda3/envs/cs294/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [ ]:
rewards = []
observations = []
for _ in range(1000):
    obs = env.reset()
    reward = 0
    done = False
    while not done:
        env.render("human")
        observations.append(obs)
        obs, r, done, _ = env.step(model.predict(obs.reshape((1,-1))).reshape((-1)))
        reward = reward + r
    rewards.append(reward)
env.close()

In [22]:
expert_data = {'observations': np.array(observations)}
with open(os.path.join(rollouts_dir, env_name + '_results.pkl'), 'wb') as f:
    pickle.dump(expert_data, f, pickle.HIGHEST_PROTOCOL)

In [23]:
print(np.mean(rewards), np.min(rewards), np.max(rewards), np.std(rewards))

625.7542174687198 -170.47798665579754 3431.8707863520162 411.7887946989022
